In [1]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [2]:
import json
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor
load_dotenv()

True

In [3]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, 
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        ))
    return kernel


In [4]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [5]:
REVIEWER_NAME = "ArtDirector"
REVIEWER_INSTRUCTIONS = """
You are an art director who has opinions about copywriting born of a love for David Ogilvy.
The goal is to determine if the given copy is acceptable to print.
If so, state that it is approved.
If not, provide insight on how to refine suggested copy without example.
"""

COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = """
You are a copywriter with ten years of experience and are known for brevity and a dry humor.
The goal is to refine and decide on the single best copy as an expert in the field.
Only provide a single proposal per response.
You're laser focused on the goal at hand.
Don't waste time with chit chat.
Consider suggestions when refining an idea.
"""

TASK = "a slogan for a new line of electric cars."

In [6]:
# 1. Create the reviewer agent based on the chat completion service
agent_reviewer = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("artdirector"),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)

In [7]:
# 2. Create the copywriter agent based on the chat completion service
agent_writer = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("copywriter"),
    name=COPYWRITER_NAME,
    instructions=COPYWRITER_INSTRUCTIONS,
)

In [13]:
# 3. Place the agents in a group chat with a custom termination strategy
group_chat = AgentGroupChat(
    agents=[
        agent_writer,
        agent_reviewer,
    ],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[agent_reviewer],
        maximum_iterations=10,
    ),
)

In [14]:
from semantic_kernel.agents.group_chat.agent_chat import ChatMessageContent

message_content = ChatMessageContent(role="user", content="We need a slogan for a new line of electric cars.")

In [15]:




# 4. Add the task as a message to the group chat
#await group_chat.add_chat_message(message=TASK)
# 4. Add the task as a message to the group chat
await group_chat.add_chat_message(message_content)
print(f"# User: {TASK}")

# User: a slogan for a new line of electric cars.


In [16]:
# 5. Invoke the chat
async for content in group_chat.invoke():
    print(f"# {content.name}: {content.content}")


# CopyWriter: "Charge Ahead."
# ArtDirector: Not approved. While "Charge Ahead" has simplicity, it feels overly generic, failing to stir distinct emotions or differentiate your brand from competitors. A slogan for electric cars should evoke innovation, environmental consciousness, and a sense of forward motion. Strive for phrasing that captures the audience’s imagination and carves out a unique space for your brand—words that carry both the promise of technology and the aspiration of sustainability.
